In [91]:
import json
import numpy as np
import pandas as pd

import MeCab

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.manifold import TSNE

import plotly.express as px
from datetime import datetime


In [92]:
stories_json = []
with open('./output_storiesv2.json', 'r') as f:
    stories_json = stories_json + json.loads(f.read())

print(len(stories_json))
print(stories_json[1].keys())

407
dict_keys(['index', 'title', 'broadcasting_date', 'abstracts_list'])


In [93]:
stories_df = pd.DataFrame(stories_json)
stories_df["abstract_joined"] = stories_df["abstracts_list"].str.join(" ")
stories_df.head()

,index,title,broadcasting_date,abstracts_list,abstract_joined
0,0325,春だ！映画だ！３時間アニメ祭り\r\n「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い...,2013年3月15日,[ 何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいな...,何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいない...
1,0326,「ばくはつコショウ」「何が何でもお花見を」[2013年4月12日放送],2013年4月12日,[ ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。この...,ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。このコ...
2,0327,「やりクリしてハワイ旅行」「しずかちゃんのはごろも」[2013年4月26日放送],2013年4月26日,[ スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとする...,スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとするが...
3,0328,「ひるねは天国で」「どくさいスイッチ」[2013年5月3日放送],2013年5月3日,[ のび太とドラえもんが部屋で昼寝をしていると、突然大きな音が聞こえてくる。となりの家が改築...,のび太とドラえもんが部屋で昼寝をしていると、突然大きな音が聞こえてくる。となりの家が改築工...
4,0329,「インスタントママ」「狙われたジャイアン」[2013年5月10日放送],2013年5月10日,[ そうじにせんたく、町内会の会合と毎日大忙しのママが、「体がもうひとつほしい」とため息をつ...,そうじにせんたく、町内会の会合と毎日大忙しのママが、「体がもうひとつほしい」とため息をつい...


In [94]:
t = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
def mecab_tokenizer(text):
    parsed_lines = t.parse(text).split("\n")[:-2]
    surfaces = [l.split('\t')[0] for l in parsed_lines]
    features = [l.split('\t')[1] for l in parsed_lines]
    # 原型を取得
    bases = [f.split(',')[6] for f in features]
    # 品詞を取得
    pos = [f.split(',')[0] for f in features]

    # 各単語を原型に変換する
    token_list = [b if b != '*' else s for s, b in zip(surfaces, bases)]
    # 品詞を絞り込み
    target_pos = ["名詞", "動詞", "形容詞"]
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    # アルファベットを小文字に統一
    token_list = [t.lower() for t in token_list]

    return [ text for text in filter(lambda x: not x.isascii(), token_list)]

In [95]:
# ストーリーが取得できたものに絞る
stories_df = stories_df[stories_df['abstract_joined'].str.len() > 0] 
sentences = stories_df["abstract_joined"].apply(mecab_tokenizer)
print(sentences)

0      [何, やる, うまい, いく, のび太, 魔法, 魔法, 使える, 便利, 楽しい, なる...
1      [朝, 寝坊, する, のび太, 泣きつく, れる, ドラえもん, 爆発, コショウ, 取り...
2      [スネ夫, ハワイ旅行, じまん, する, れる, のび太, 自分, ハワイ, 行く, ママ...
3      [のび太, ドラえもん, 部屋, 昼寝, する, いる, 音, 聞こえる, くる, となり,...
4      [そうじ, せんたく, 町内会, 会合, 毎日, 大忙し, ママ, 体, ひとつ, ほしい,...
                             ...                        
402    [しずか, 遊び, さそえる, ジャイアン, 意味, ない, ボコボコ, する, れる, し...
403    [スネ夫, 手, 入る, 恐竜, きょう, りゅう, プラモデル, じまん, する, れる,...
404    [秋, 楽しむ, ため, 別荘, べっそう, 行く, スネ夫, 季節感, ない, 顔, バカ...
405    [空き地, サッカー, する, 遊ぶ, いる, ジャイアン, スネ夫, たち, ジャイアン,...
406    [使い, の, ちゅ, 腕木, げん, 悪い, そう, ジャイアン, 見かける, 別, 道,...
Name: abstract_joined, Length: 407, dtype: object


In [96]:
abstract_list = sentences.tolist()
trainings = [TaggedDocument(words = data, tags = [i]) for i,data in enumerate(abstract_list)]
m = Doc2Vec(documents= trainings, dm = 1, window=8, min_count=10, workers=4)


In [97]:
vector_list = m.dv.vectors
tsne = TSNE(n_components=2)
vector_enbedded = tsne.fit_transform(vector_list)
print(vector_enbedded)

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:805: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



[[ 1.64828091e+01  5.31324100e+00]
 [ 6.39169025e+00  4.95657825e+00]
 [ 1.39054394e+01  3.45471668e+00]
 [ 2.37607441e+01  1.19673376e+01]
 [ 7.65951157e+00  4.44239044e+00]
 [ 2.21130123e+01  1.23943520e+01]
 [ 2.13880367e+01  1.16292648e+01]
 [ 2.37712345e+01  1.51009693e+01]
 [ 2.09282284e+01  1.50846930e+01]
 [ 1.14479170e+01  1.10665970e+01]
 [ 1.18734140e+01  1.40172553e+00]
 [-7.92190361e+00  5.40205812e+00]
 [-1.66754761e+01 -4.33327579e+00]
 [ 5.96730900e+00  1.30084524e+01]
 [ 1.47331929e+00 -3.67356372e+00]
 [ 8.50683498e+00  5.93554831e+00]
 [ 7.56749201e+00  5.22509766e+00]
 [ 7.31700945e+00  8.47035122e+00]
 [-1.46511383e+01  6.03860760e+00]
 [ 1.33259754e+01  3.89183283e+00]
 [ 8.07059288e+00  1.41432409e+01]
 [ 7.63798285e+00  7.99288750e-01]
 [ 1.90589027e+01  1.58749876e+01]
 [-1.18657017e+00  7.85830212e+00]
 [ 1.39159527e+01  1.92199528e+00]
 [-1.54916563e+01 -8.73367310e-01]
 [-1.33666430e+01 -7.63101101e+00]
 [-2.22475481e+00 -1.00289297e+01]
 [ 3.74444222e+00  1

In [98]:
vector_df = pd.DataFrame.from_dict({
    'vector_x': [v[0] for  v in vector_enbedded],
    'vector_y': [v[1] for  v in vector_enbedded]
})
stories_df_merged = stories_df.join(vector_df)
stories_df_merged['broadcasting_dt'] = stories_df_merged['broadcasting_date'].apply(lambda d: datetime.strptime(d, '%Y年%m月%d日'))
stories_df_merged['broadcasting_year'] = stories_df_merged['broadcasting_dt'].dt.year

In [99]:
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', color='broadcasting_year', hover_data=["index", "title"])
fig.show()

In [100]:
df = px.data.tips()
fig = px.scatter(stories_df_merged, x='vector_x', y='vector_y', hover_data=["index", "title"], facet_row="broadcasting_year", width=800, height=2000)
fig.show()